## Data Profiling

In [1]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../../../..')))

In [2]:
from data_sc.src.utils.db import get_db_connection

# Call the function
conn = get_db_connection()

Database connection established successfully (using .env).


### 1. Load Data from Database

We use raw SQL to query the `legislative_proposal` table and load it into a DataFrame.

In [ ]:
import pandas as pd

# Load table into DataFrame
query = "SELECT * FROM legislative_proposal"
df = pd.read_sql(query, conn)

df.head()

### 2. Missing Values Overview

Check which fields have missing/null values to assess data completeness.


In [18]:
# Get null counts
null_counts = df.isnull().sum().sort_values(ascending=False)

# Convert to DataFrame for formatting
null_df = null_counts.to_frame(name='missing_count')

# Add total row
total_missing = null_df['missing_count'].sum()
null_df.loc['TOTAL'] = total_missing

# Display
display(null_df)

,missing_count
first_senate_registration_number,7632
matching_title,7616
status_senate,6080
status_cdep,5436
deadline,5280
opinion,4932
government_registration_number,346
idp,285
law_character,24
senate_registration_number,17


### 3. Duplicate Checks & ID Uniqueness

Check for duplicates in `normalized_title` and validate uniqueness of `idp`.


In [5]:
df['normalized_title'].value_counts().head(10)

normalized_title
propunere legislativa pentru modificarea si completarea legii educatiei nationale nr.1/2011                                                 38
propunere legislativa pentru modificarea si completarea legii nr.95/2006 privind reforma in domeniul sanatatii                              23
proiect de lege privind abilitarea guvernului de a emite ordonante                                                                          21
proiect de lege pentru modificarea si completarea ordonantei de urgenta a guvernului nr.195/2002 privind circulatia pe drumurile publice    16
propunere legislativa pentru modificarea si completarea legii nr.1/2011 a educatiei nationale                                               16
propunere legislativa pentru modificarea si completarea legii nr.227/2015 privind codul fiscal                                              15
propunere legislativa pentru modificarea legii nr.227/2015 privind codul fiscal                                              

In [6]:
# Unique vs total count of idp
print("Unique idp:", df['idp'].nunique())
print("Total rows:", len(df))

Unique idp: 7591
Total rows: 7876


### 5. Date Consistency: `year_issue` vs `latest_procedure_date`

Ensure the `latest_procedure_date` aligns reasonably with `year_issue`.

In [12]:
df['latest_procedure_date'] = pd.to_datetime(df['latest_procedure_date'])
df['procedure_year'] = df['latest_procedure_date'].dt.year

df[['year_issue', 'procedure_year']].dropna().sample(5)

,year_issue,procedure_year
7096,2017.0,2025
76,2021.0,2022
1401,2023.0,2023
3279,2014.0,2015
2952,2020.0,2022


## 🔎 Flagging Legislative Proposals Based on Year Differences

To better understand the legislative processing time, we add a `year_flag` column:

- **"same_year"** – if the proposal and latest procedure occurred in the same year.
- **"delayed"** – if the procedure happened in a later year.
- **"inconsistent"** – if procedure year is earlier than the issued year (could be data error or irregularity).
- **"unknown"** – One or both of the `year_issue` or `procedure_year` values are missing, making it impossible to evaluate.

In [13]:
def flag_year_relation(row):
    if pd.isna(row['year_issue']) or pd.isna(row['procedure_year']):
        return 'unknown'
    elif row['year_issue'] == row['procedure_year']:
        return 'same_year'
    elif row['year_issue'] < row['procedure_year']:
        return 'delayed'
    else:
        return 'inconsistent'

df['year_flag'] = df.apply(flag_year_relation, axis=1)
df['year_flag'].value_counts()

year_flag
delayed         4813
same_year       3049
unknown           12
inconsistent       2
Name: count, dtype: int64

In [17]:
# Filter rows with flag 'unknown' and show only relevant columns
df_unknown = df[df['year_flag'] == 'unknown'][['year_issue', 'procedure_year']]

# Filter rows with flag 'inconsistent' and show only relevant columns
df_inconsistent = df[df['year_flag'] == 'inconsistent'][['year_issue', 'procedure_year']]

# Display samples
print("📌 Unknown proposals:")
display(df_unknown.head(15))

print("⚠️ Inconsistent proposals:")
display(df_inconsistent.head())


📌 Unknown proposals:


,year_issue,procedure_year
371,NaN,2014
1783,NaN,2017
2549,NaN,2019
2576,NaN,2019
3273,NaN,2020
3748,NaN,2015
4500,NaN,2023
4792,NaN,2014
5449,NaN,2019
5581,NaN,2016


⚠️ Inconsistent proposals:


,year_issue,procedure_year
638,2022.0,2021
3860,2020.0,2019


### 6. Boolean Flags Distribution

Check the consistency and distribution of flags like `active`, `published`, `senate_active`, `cdep_active`.


In [9]:
df[['active', 'published']].value_counts()


active  published
False   True         3354
True    False        2502
False   False        2020
Name: count, dtype: int64

In [10]:
df[['senate_active', 'cdep_active']].value_counts()

senate_active  cdep_active
False          False          5392
               True           2235
True           False           226
               True             23
Name: count, dtype: int64